# Event Data Analysis & Transformation

## Table Of Contents

* [Business Problem Statement](#business-problem-statement)
* [Objectives](#objectives)
* [Data Description](#data-description)
* [Data Exploration](#data-exploration)
* [DynamicPriceOption](#dynamicpriceoption)
* [DynamicPriceRange](#dynamicpricerange)
* [CurratedOfferoptions](#curratedofferoptions)

<a name="business-problem-statement"></a>
# Business Problem Statement:

The organization encounters a challenge in structuring event data received in JSON format into tables. The objective is to convert the provided "case.json" file into three distinct CSV files, adhering to specific formatting rules for each file. The transformation involves creating tables for CuratedOfferOptions, DynamicPriceOption, and DynamicPriceRange.

1. CuratedOfferOptions.csv:

    CurationProvider, OfferId, DealerId, UniqueOptionId, OptionId: Enclosed in quotes.
    IsMobileDealer, IsOpen, ChamaScore, MinimumPrice, MaximumPrice, DynamicPrice, FinalPrice: Without quotes.
    DefeatPrimaryReason, DefeatReasons: Enclosed in quotes.
    Eta, EnqueuedTimeSP: Enclosed in quotes and converted to Brazilian timezone (UTC-3) in DD/MM/YYYY format.
    
2. DynamicPriceOption.csv:

    Provider, OfferId, UniqueOptionId: Enclosed in quotes.
    BestPrice, EnqueuedTimeSP: Without quotes.
    
2. DynamicPriceRange.csv:

    Provider, OfferId: Enclosed in quotes.
    MinGlobal, MinRecommended, MaxRecommended, DifferenceMinRecommendMinTheory, EnqueuedTimeSP: Without quotes.
    
The initial step is to explore the provided "case.json" file and comprehend its structure. Understanding the JSON data will aid in devising an effective strategy for the subsequent transformations. This exploration phase ensures that the solution is tailored to the specific needs of the company and that the resulting CSV files meet the outlined requirements.

<div style="text-align:center">
    <img src="https://d2908q01vomqb2.cloudfront.net/f6e1126cedebf23e1463aee73f9df08783640400/2021/07/16/BlogIngestData.png" alt="Image Alt Text" />
</div>


<a name="objectives"></a>
# Objectives

The objective is to process and transform JSON data into three distinct CSV files, namely CuratedOfferOptions.csv, DynamicPriceOption.csv, and DynamicPriceRange.csv, adhering to specific formatting rules outlined in the task.

<a name="data-description"></a>
# Data Description:

Data Description:

The dataset is provided in a file named case.json, and each record within this file follows a specific format, as illustrated by the example below:

1. **EnqueuedTimeUtc:** Indicates the timestamp of the event in the UTC timezone (e.g., "2021-09-05 08:04:08 UTC").

2. **EventName:** Describes the type of event (e.g., "DynamicPrice_Result").

3. **Payload:** Contains a JSON-formatted string encapsulated within double quotes, representing additional details related to the event. The example shows a snippet where the provider is "ApplyDynamicPriceRange," and an associated offerId is provided.


<a name="data-exploration"></a>
# Data Exploration:
Import neccessary libraries and  Read JSON file into a DataFrame

In [2]:
import pandas as pd
import json

# Read JSON file into a DataFrame
data_frame = pd.read_json(path_or_buf="case.json")

 Sampling 10 rows with random state for reproducibility

In [4]:
# Sample 10 rows with random state for reproducibility
sampled_data_frame = data_frame.sample(n=10, random_state=42)

Extracting 'EnqueuedTimeUtc' from dataframe & transofrming it into brazillion timezoe

In [5]:
# Transform to Brazilian timezone
sampled_data_frame["EnqueuedTimeUtc"] = pd.to_datetime(arg=sampled_data_frame["EnqueuedTimeUtc"]).dt.tz_convert(tz="Brazil/East")


<a name="dynamicpriceoption"></a>
# DynamicPriceOption

selecting rows from the original DataFrame(sampled_data_frame) where the "EventName" column has the value "DynamicPrice_Result" and it creates a new DataFrame (dynamic_price_events) containing only these selected rows

In [7]:
# Select events related to dynamic price
dynamic_price_events = sampled_data_frame[sampled_data_frame["EventName"] == "DynamicPrice_Result"]

the function 'transform_dynamic_price_option' processes a DataFrame row containing dynamic price option data, extracts relevant information, and organizes it into a structured format for further analysis or storage.








In [8]:
# Extract dynamic price option information
def transform_dynamic_price_option(row):
    """Transform the JSON payload into a dynamic price option dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []

    if payload_row["provider"] == "ApplyDynamicPricePerOption":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        
        for unique_option in payload_row["algorithmOutput"]:
            price_option = {
                "Provider": provider,
                "OfferId": offer_id,
                "UniqueOptionId": f"\"{unique_option['uniqueOptionId']}\"",
                "BestPrice": unique_option["bestPrice"],
                "EnqueuedTimeSP": enqueued_time
            }
            result.append(price_option)
    
    return result

code is transforming each row of the dynamic_price_events DataFrame using the transform_dynamic_price_option function, and the resulting list of dictionaries is stored in dynamic_price_option_list. Each dictionary in this list represents the structured transformation of dynamic price option data from the corresponding row in the original DataFrame.

In [11]:
# Apply the function to the DataFrame
dynamic_price_option_list = dynamic_price_events.apply(transform_dynamic_price_option, axis=1)

This process effectively flattens the nested structure, making it a one-dimensional list of dictionaries containing dynamic price option data.

In [12]:
# Flatten the list of dictionaries
flattened_dynamic_price_option = [x for xs in dynamic_price_option_list for x in xs]

creating a Pandas DataFrame from the previously flattened list , which contains information about dynamic price options. Subsequently, the DataFrame is being saved to a CSV file 

In [13]:
# Create DataFrame from the flattened list and save to CSV
dynamic_price_option_df = pd.DataFrame(flattened_dynamic_price_option)
dynamic_price_option_df.to_csv("transformed_option_payload.csv", index=False)

<a name="dynamicpricerange"></a>
# DynamicPriceRange

This code defines a function transform_dynamic_price_range that takes a row from a DataFrame, processes the JSON payload in the row, and transforms it into a dynamic price range DataFrame. The resulting DataFrame contains information about the dynamic price range, including provider, offer ID, minimum global and recommended prices, maximum recommended price, the difference between minimum recommended and theoretical minimum prices, and the enqueued time in a specific format. The function returns a list containing this information for further processing

In [14]:
# Extract dynamic price range information
def transform_dynamic_price_range(row):
    """Transform the JSON payload into a dynamic price range dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []

    if payload_row["provider"] == "ApplyDynamicPriceRange":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        algorithm_output = payload_row["algorithmOutput"]
        
        price_range = {
            "Provider": provider,
            "OfferId": offer_id,
            "MinGlobal": algorithm_output["min_global"],
            "MinRecommended": algorithm_output["min_recommended"],
            "MaxRecommended": algorithm_output["max_recommended"],
            "DifferenceMinRecommendMinTheory": algorithm_output["differenceMinRecommendMinTheory"],
            "EnqueuedTimeSP": enqueued_time
        }
        result.append(price_range)

    return result

 applying the transform_dynamic_price_range function to each row in the DataFrame along the specified axis . It creates a list  containing the transformed dynamic price range information for each row in the DataFrame.

In [15]:
# Apply the function to the DataFrame
dynamic_price_range_list = dynamic_price_events.apply(transform_dynamic_price_range, axis=1)

flattens the list of dictionaries  into a single list. It concatenates the dictionaries from each row into one flat list.

In [18]:

# Flatten the list of dictionaries
flattened_dynamic_price_range = [x for xs in dynamic_price_range_list for x in xs]

 creates a DataFramefrom the flattened list of dictionaries and then saves it to a CSV file 

In [29]:
# Create DataFrame from the flattened list and save to CSV
dynamic_price_range_df = pd.DataFrame(flattened_dynamic_price_range)
dynamic_price_range_df.to_csv("transformed_price_payload.csv", index=False)

<a name="curratedofferoptions"></a>
# CurratedOfferOptions

 filtering events in the DataFrame to include only those related to curated offers by selecting rows where the "EventName" column is equal to "CurateOffer_Result".

In [20]:

# Select events related to curated offers
curated_offers = sampled_data_frame[sampled_data_frame["EventName"] == "CurateOffer_Result"]

This function extracts information from the JSON payload in the "Payload" column of a DataFrame row. The extracted information is related to curated offers, and the function returns a list of dictionaries containing the transformed data. The data includes details about curation providers, offer IDs, dealer IDs, and various properties of curated offer options. If available, defeat reasons are also included in the result.

In [21]:

# Extract curated offer information
def transform_curated_offer(row):
    """Transform JSON payload into curated offer dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []

    for el in payload_row:
        curation_provider = f"\"{el['curationProvider']}\""
        offer_id = f"\"{el['offerId']}\""
        dealer_id = f"\"{el['dealerId']}\""

        for option in el["options"]:
            offer_options = {
                "CurationProvider": curation_provider,
                "OfferId": offer_id,
                "DealerId": dealer_id,
                "UniqueOptionId": f"\"{option['uniqueOptionId']}\"",
                "OptionId": f"\"{option['optionId']}\"",
                "IsMobileDealer": option["isMobileDealer"],
                "IsOpen": option["isOpen"],
                "Eta": f"\"{option['eta']}\"",
                "ChamaScore": option["chamaScore"],
                "ProductBrand": f"\"{option['productBrand']}\"",
                "IsWinner": option["isWinner"],
                "MinimumPrice": option["minimumPrice"],
                "MaximumPrice": option["maximumPrice"],
                "DynamicPrice": option["dynamicPrice"],
                "FinalPrice": option["finalPrice"]
            }

            if "defeatPrimaryReason" in option:
                offer_options["DefeatPrimaryReason"] = f"\"{option['defeatPrimaryReason']}\""
                offer_options["DefeatReasons"] = f"\"{option['defeatReasons']}\""

            result.append(offer_options)

    return result


 applying transform_curated_offer function to each row of the DataFrame curated_offers to generates a list of dictionaries containing transformed information about curated offers for each row in the DataFrame.

In [22]:
# Apply the function to the DataFrame
curated_offer_list = curated_offers.apply(transform_curated_offer, axis=1)

flattening list of dictionaries into a single list which represents the transformed information about curated offers.

In [23]:
# Flatten the list of dictionaries
flattened_curated_offer = [x for xs in curated_offer_list for x in xs]

creating DataFrame from the flattened list of dictionaries, which contains information about curated offers.

In [24]:

# Create DataFrame from the flattened list
curated_offer_df = pd.DataFrame(flattened_curated_offer)

Saving the curated offer DataFrame to a CSV file without including the index column.

In [30]:

# Save to CSV
curated_offer_df.to_csv("curated_options_output.csv", index=False)
